In [ ]:
from pymongo import MongoClient
import numpy as np
import base64
from cpsdriver.codec import DocObjectCodec
import matplotlib.pyplot as plt
import matplotlib.dates as md

from pymongo import MongoClient
import numpy as np
import base64
# from cpsdriver.codec import DocObjectCodec
import cpsdriver.codec as codec
from datetime import datetime
from WeightTrigger import WeightTrigger as WT
import BookKeeper as BK
import math_utils
import math
from ScoreCalculate import *

from utils import *
from config import *


In [ ]:
def plot_data2(mean, std, timestamps, title=None):
    assert len(mean) == len(timestamps)
    assert len(std) == len(timestamps)

    date_times = []
    for timestamp in timestamps:
        date_times.append(datetime.fromtimestamp(timestamp))
    fig = plt.figure(figsize=(16,4))
    fig.suptitle(title)
    
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.title.set_text('mean')
    ax2.title.set_text('std')
    
    ax1.plot(date_times, mean)
    ax2.plot(date_times, std)
    
    for tick in ax1.get_xticklabels():
        tick.set_rotation(75)
    for tick in ax2.get_xticklabels():
        tick.set_rotation(75)
        
    xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
    ax1.xaxis.set_major_formatter(xfmt)
    ax1.get_yaxis().get_major_formatter().set_useOffset(False)
    ax2.xaxis.set_major_formatter(xfmt)
    ax2.get_yaxis().get_major_formatter().set_useOffset(False)
    
    ax1.set_ylabel('weight / g')
    ax1.set_xlabel('time / one data point per 0.2s')
    ax2.set_ylabel('weight / g')
    ax2.set_xlabel('time / one data point per 0.2s')
    
    plt.show()

In [ ]:
dbName="BASELINE-7"
myBK = BK.BookKeeper(dbName)
weightTrigger = WT(myBK)
weight_shelf_mean, weight_shelf_std, weight_plate_mean, weight_plate_std = weightTrigger.get_moving_weight()

number_gondolas = len(weight_shelf_mean)
# reduce timestamp 
timestamps = weightTrigger.get_agg_timestamps()
for i in range(number_gondolas):
    timestamps[i] = timestamps[i][30:-29]

# sanity check
for i in range(number_gondolas):
    # weight_shelf_mean: [gondola, shelf, timestamp]
    # weight_shelf_std: [gondola, shelf, timestamp]
    # weight_plate_mean: [gondola, shelf, plate, timestamp]
    # weight_plate_std: [gondola, shelf, plate, timestamp]
    # timestamps: [gondola, timestamp]
    timestamps_count = len(timestamps[i])
#     print ('weight_shelf_mean', weight_shelf_mean[i].shape)
#     print ('weight_shelf_std',  weight_shelf_std[i].shape)
#     print ('weight_plate_mean', weight_plate_mean[i].shape)
#     print ('weight_plate_std', weight_plate_std[i].shape)
#     print ('timestamps', timestamps_count)
    assert (timestamps_count == weight_shelf_mean[i].shape[1])
    assert (timestamps_count == weight_shelf_std[i].shape[1])
    assert (timestamps_count == weight_plate_mean[i].shape[2])
    assert (timestamps_count == weight_plate_std[i].shape[2])
#     print ('timestamps', timestamps[i][0], "to", timestamps[i][-1])



In [ ]:
# take water
plot_gondola_id = 1
plot_shelf_id = 3


title = '[gondola %d shelf %d]' % (plot_gondola_id, plot_shelf_id)
plot_data2(weight_shelf_mean[plot_gondola_id-1][plot_shelf_id-1],
          weight_shelf_std[plot_gondola_id-1][plot_shelf_id-1], 
          timestamps[plot_gondola_id-1], 
          title)

# plot_plate_ids = [1,2,3,4,5,6,7,8,9,10,11,12]
# for plot_plate_id in plot_plate_ids:
#     title = '[gondola %d shelf %d plate %d]' % (plot_gondola_id, plot_shelf_id, plot_plate_id)
#     plot_data2(weight_plate_mean[plot_gondola_id-1][plot_shelf_id-1][plot_plate_id-1],
#               weight_plate_std[plot_gondola_id-1][plot_shelf_id-1][plot_plate_id-1], 
#               timestamps[plot_gondola_id-1], 
#               title)


In [ ]:
events = weightTrigger.detect_weight_events(weight_shelf_mean, weight_shelf_std, weight_plate_mean, weight_plate_std, timestamps)
for event in events:
    print (event, '\n')